In [18]:
import pandas as pd

df_all = pd.read_csv('data/processed/all_topics.csv')

# Update PDF url
df_all['pdf_link'] = df_all['pdf_link'].apply(lambda x: 'https://api.openreview.net' + x if  isinstance(x, str) else x)

# Add google scholar link by title
df_all['google_scholar_link'] = df_all['title'].apply(lambda x: 'https://scholar.google.com/scholar?q=' + x.replace(' ', '+'))

# Remove duplicates by title
df_all = df_all.drop_duplicates(subset=['title'])
len(df_all), df_all.columns

(10590,
 Index(['title', 'year', 'source', 'authors', 'class', 'keywords', 'abstract',
        'pdf_link', 'topic', 'google_scholar_link'],
       dtype='object'))

# Multi-modal

In [19]:
def filter_by_topic(df, topic):
    return df[df['topic'] == topic]

In [20]:
df_multimodal = filter_by_topic(df_all, 'multimodal')
df_multimodal.to_csv('data/topics/multimodal.csv', index=False)
len(df_multimodal)

44

# Diffusion models

In [21]:
df_diffm = filter_by_topic(df_all, 'diffusion models')
df_diffm.to_csv('data/topics/diffm.csv', index=False)
len(df_diffm)

35

# Graph

In [22]:
df_graph = filter_by_topic(df_all, 'graph')
df_graph.to_csv('data/topics/graph.csv', index=False)
len(df_graph)

519

# 3D

In [24]:
df_3d = filter_by_topic(df_all, '3d')
df_3d.to_csv('data/topics/3d.csv', index=False)
len(df_3d)

62

# LLM

In [25]:
df_llm = filter_by_topic(df_all, 'llm')
df_llm.to_csv('data/topics/llm.csv', index=False)
len(df_llm)

67